In [308]:
import torch
import pandas as pd


import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm
import torchvision.models as models
from scipy.stats import wasserstein_distance


In [309]:
hadron_level_data = pd.read_csv("hadron_level_events.csv")
parton_level_data = pd.read_csv("parton_level_events.csv")

In [310]:
hadron_level_data = hadron_level_data.drop(columns=["Unnamed: 91"]).dropna()
parton_level_data = parton_level_data.drop(columns=["Unnamed: 61"]).dropna()

In [311]:
parton_level_data.shape

(8649, 61)

In [312]:
x = hadron_level_data.values
y = parton_level_data.values

In [313]:
x.shape

(8649, 91)

In [314]:
scx = StandardScaler()
x = scx.fit_transform(x)

(91,)

In [315]:
scy = StandardScaler()
y = scy.fit_transform(y)

(8649, 61)

In [316]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [317]:
class dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.length = self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return self.length

In [318]:
class Net(nn.Module):
    def __init__(self, input_shape):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_shape, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 61)

    def forward(self,x):
        x = self.fc1(x).relu()
        x = self.fc2(x).relu()
        x = self.fc3(x).relu()
        x = self.fc4(x).relu()
        x = self.fc5(x)
        return x

In [319]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.99, random_state=21, shuffle=False)

In [320]:
# X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Y_train = Y_train.astype('float32')
Y_test = Y_test.astype('float32')

In [321]:
# X_train = torch.from_numpy(X_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
# Y_train = torch.from_numpy(Y_train).to(device)
Y_test = torch.from_numpy(Y_test).to(device)

In [322]:
test_set = dataset(X_test, Y_test)

/var/folders/9k/5mfcgwhd61z19vw5lpk_0kw80000gn/T/ipykernel_30418/3780249379.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x, dtype=torch.float32)
/var/folders/9k/5mfcgwhd61z19vw5lpk_0kw80000gn/T/ipykernel_30418/3780249379.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.float32)


In [324]:
model = Net(input_shape=x.shape[1])
model.load_state_dict(torch.load('model.pth', map_location=torch.device('cpu')))


/var/folders/9k/5mfcgwhd61z19vw5lpk_0kw80000gn/T/ipykernel_30418/3625934217.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth',

<All keys matched successfully>

In [333]:
predicted = model(X_test)
predicted = scy.inverse_transform(predicted.detach().cpu().numpy())

In [334]:
actual = scy.inverse_transform(Y_test.cpu().numpy())

In [330]:
predicted = predicted[0][1:]#.reshape(20,3)
predicted.shape

(60,)

In [331]:
actual = actual[0][1:].reshape(20,3)

In [332]:
actual.shape
acutal[0].shape

(3,)

In [266]:
real_actual = np.array([np.array(row) for row in actual])
real_predicted = np.array([np.array(row) for row in predicted])

[[ 1.0702400e+03 -7.3032908e-02  2.5971298e+00]
 [ 1.3230203e+00 -7.4330002e-01  2.5846000e+00]
 [ 3.8378203e+00 -5.5901402e-01  2.4090600e+00]
 [ 1.3745898e+00  3.4882399e-01  2.6781299e+00]
 [ 3.6005858e-07 -5.3891679e-11  1.1386213e-09]
 [ 2.8686350e-07 -1.2742002e-10 -5.3570953e-10]
 [ 1.5708395e-08 -2.1051949e-10  2.2917593e-11]
 [ 3.6678660e-08 -1.3598924e-11 -4.5976200e-12]
 [ 4.9224251e-08  2.3911040e-11  4.9870491e-10]
 [ 3.6454575e-09 -2.5246456e-11  5.9759741e-12]
 [-4.8031392e-09  7.7427793e-11  6.6660080e-11]
 [ 1.1996639e-08  8.2607685e-11  2.0929888e-11]
 [ 3.8353276e-09  7.2147940e-11 -7.0367170e-11]
 [ 2.1153125e-08  1.7033312e-11 -8.9261237e-11]
 [ 5.2050630e-09  6.1453523e-11  1.1742382e-10]
 [-4.6975890e-09  3.8900657e-11  1.6942697e-13]
 [-4.5489817e-10  2.0656714e-11 -1.8877508e-12]
 [-1.2217279e-09  2.8144460e-13  1.2747237e-10]
 [-6.4216665e-10 -4.9043804e-11  4.4416137e-11]
 [-7.7084041e-11 -1.0455968e-11  1.9670019e-12]]


In [267]:
# emd = wasserstein_distance(real_actual, real_predicted)

ValueError: object too deep for desired array

In [272]:
values = []

for event in predicted.shape[0]:
    emdval, G = ef.emd.emd(predicted[event, :], actual[event, :], R=1, return_flow=True)
    values.append(emdval)

average = np.mean(values)

if(average < 5): print(f'Average EMD value is {average}, which is in a good range')
elif(average < 6): print(f'Average EMD value is {average}, which is in a an okay range')
else: print(f'Average EMD value is {average}, bad accuracy')

Average EMD value is 4, which is in a good range
